# 使用BDMLtools做策略分析报告 <font size=3 >by 曾珂</font>

In [1]:
import scorecardpy as sc
import BDMLtools as bm
import pandas as pd
import numpy as np

+ load germancredit data

In [2]:
dt=sc.germancredit().copy()
dt['creditability']=dt['creditability'].map({'good':0,'bad':1})

+ 将germancredit的数据类型进行指定
    + num列-->float64
    + str列-->object
    + date列-->datetime

In [3]:
dtypes_dict={
    'num':['age.in.years',
         'credit.amount',
         'creditability',
         'duration.in.month',
         'installment.rate.in.percentage.of.disposable.income',
         'number.of.existing.credits.at.this.bank',
         'number.of.people.being.liable.to.provide.maintenance.for',
         'present.residence.since'],
    'str':['housing','telephone','foreign.worker','purpose','job','personal.status.and.sex','property',
           'credit.history','savings.account.and.bonds','present.employment.since',
           'status.of.existing.checking.account',
           'other.installment.plans','other.debtors.or.guarantors'],
    'date':[]
}

In [4]:
da=bm.dtypeAllocator(dtypes_dict=dtypes_dict).fit(dt)
dt=da.transform(dt)
X=dt.drop('creditability',axis=1)
y=dt['creditability']

## BDMLtools中的报告

实践进行策略分析时，有时希望快速生成策略特征的分析报告,BDMLtools提供了三种特征分析库:
    
+ varReport:特征分析报告
+ varReportSinge:单特征分析报告
+ varGroupsReport:组特征分析报告

## varReport

### 基本用法

一般特征分析需先进行分箱再生成报告,这里使用binSelector进行最优分箱

In [5]:
bin_tree=bm.binSelector(method='tree',bin_num_limit=8,n_jobs=1,iv_limit=0).fit(X,y)

+ bin_tree中的属性breaks_list为分箱字典,包含了所有特征的分箱结果,格式与scorecardpy一致
+ 将breaks_list作为varReport的入参以按照指定的breaks_list分箱产生特征分析报告

In [6]:
vtab=bm.varReport(bin_tree.breaks_list,n_jobs=1).fit(X,y)

+ vtab的var_report_dict保存了所有特征的分箱结果,结构与scorecardpy一致

In [7]:
vtab.var_report_dict.keys()

dict_keys(['age.in.years', 'credit.amount', 'credit.history', 'duration.in.month', 'foreign.worker', 'housing', 'installment.rate.in.percentage.of.disposable.income', 'job', 'number.of.existing.credits.at.this.bank', 'number.of.people.being.liable.to.provide.maintenance.for', 'other.debtors.or.guarantors', 'other.installment.plans', 'personal.status.and.sex', 'present.employment.since', 'present.residence.since', 'property', 'purpose', 'savings.account.and.bonds', 'status.of.existing.checking.account', 'telephone'])

In [8]:
vtab.var_report_dict['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,105,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,60,0.060,39.0,21.0,0.350000,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,242,0.242,182.0,60.0,0.247934,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,80,0.080,66.0,14.0,0.175000,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,58,0.058,31.0,27.0,0.465517,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,138,0.138,89.0,49.0,0.355072,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


这里每一个报表中bin索引,
+ 存在missing行用于标示缺失值,数据中的np.nan将被计算进缺失值行中,若数据没有缺失值则为默认值
+ special行用于标示除缺失值外的特殊编码,可通过special_values参数进行指定,若数据没有特殊值则为默认值,详情请见文档

In [9]:
pd.concat(vtab.var_report_dict)

variable  \
                                    bin                                                                             
age.in.years                        [-inf, 26.0)                                                     age.in.years   
                                    [26.0, 28.0)                                                     age.in.years   
                                    [28.0, 30.0)                                                     age.in.years   
                                    [30.0, 35.0)                                                     age.in.years   
                                    [35.0, 37.0)                                                     age.in.years   
...                                                                                                           ...   
status.of.existing.checking.account missing                                   status.of.existing.checking.account   
telephone                           yes, registered under the customers name                            telephone   
                                    none                                                                telephone   
                                    special                                                             telephone   
                                    missing                                                             telephone   

                                                                              count  \
                                    bin                                               
age.in.years                        [-inf, 26.0)                                190   
                                    [26.0, 28.0)                                101   
                                    [28.0, 30.0)                                 80   
                                    [30.0, 35.0)                                177   
                                    [35.0, 37.0)                                 79   
...                                                                             ...   
status.of.existing.checking.account missing                                       0   
telephone                           yes, registered under the customers name    404   
                                    none                                        596   
                                    special                                       0   
                                    missing                                       0   

                                                                              count_distr  \
                                    bin                                                     
age.in.years                        [-inf, 26.0)                                    0.190   
                                    [26.0, 28.0)                                    0.101   
                                    [28.0, 30.0)                                    0.080   
                                    [30.0, 35.0)                                    0.177   
                                    [35.0, 37.0)                                    0.079   
...                                                                                   ...   
status.of.existing.checking.account missing                                         0.000   
telephone                           yes, registered under the customers name        0.404   
                                    none                                            0.596   
                                    special                                         0.000   
                                    missing                                         0.000   

                                                                               good  \
                                    bin                                               
age.in.years                        [-inf, 26.0)                   

使用pd.concat形成所有特征的特征分析报告,与scorecardpy一致

### 样本权重

若数据是经过抽样获取的，一般希望通过样本加权建模以利于还原其违约概率

+ 加入样本权重后各个分箱的坏样本率会产生一定变化
+ 加入样本权重后各个分箱的iv,ks也会产生变化(若只对好坏样本加权且breaks一致则无变化,因为好坏分布未变化)

BDMLtools的报告支持加入样本权重

In [10]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index) #假定数据经过抽样后好样本权重为10,坏样本权重为1
sample_weight_oth=pd.Series(np.random.randint(0,100,y.size)/100,index=y.index) #模拟复杂抽样情形下的样本权重

In [11]:
vtab_ws=bm.varReport(bin_tree.breaks_list,n_jobs=1,sample_weight=sample_weight).fit(X,y)
vtab_ws_oth=bm.varReport(bin_tree.breaks_list,n_jobs=1,sample_weight=sample_weight_oth).fit(X,y)

In [12]:
vtab_ws.var_report_dict['credit.amount'] #加权后(仅类加权)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,1932,0.264658,1850.0,82.0,0.042443,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,888,0.121644,870.0,18.0,0.020270,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,411,0.056301,390.0,21.0,0.051095,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,1880,0.257534,1820.0,60.0,0.031915,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,674,0.092329,660.0,14.0,0.020772,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,337,0.046164,310.0,27.0,0.080119,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,939,0.128630,890.0,49.0,0.052183,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,239,0.032740,210.0,29.0,0.121339,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


In [13]:
vtab_ws_oth.var_report_dict['credit.amount'] #加权后(样本加权)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,122.68,0.250664,83.67,39.01,0.317982,0.100792,0.002598,0.302264,0.025772,0.15891,1400.0
"[1400.0, 1800.0)",credit.amount,54.50,0.111356,45.06,9.44,0.173211,-0.699185,0.046029,0.302264,0.040061,0.15891,1800.0
"[1800.0, 2000.0)",credit.amount,32.24,0.065874,19.08,13.16,0.408189,0.492396,0.017361,0.302264,0.004802,0.15891,2000.0
"[2000.0, 3400.0)",credit.amount,119.37,0.243901,93.80,25.57,0.214208,-0.435891,0.041958,0.302264,0.101060,0.15891,3400.0
"[3400.0, 4000.0)",credit.amount,40.96,0.083691,34.72,6.24,0.152344,-0.852481,0.049315,0.302264,0.158910,0.15891,4000.0
"[4000.0, 5000.0)",credit.amount,29.18,0.059622,15.10,14.08,0.482522,0.793915,0.042203,0.302264,0.105751,0.15891,5000.0
"[5000.0, 9200.0)",credit.amount,62.98,0.128683,41.60,21.38,0.339473,0.198210,0.005250,0.302264,0.079264,0.15891,9200.0
"[9200.0, inf)",credit.amount,27.51,0.056209,11.26,16.25,0.590694,1.230691,0.097549,0.302264,0.000000,0.15891,inf
special,credit.amount,0.00,0.000000,0.00,0.00,NaN,0.000000,0.000000,0.302264,0.000000,0.15891,special


In [14]:
vtab.var_report_dict['credit.amount'] #未加权

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.219101,9.047619e-03,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,105,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.219101,5.523810e-02,0.148571,1800.0
"[1800.0, 2000.0)",credit.amount,60,0.060,39.0,21.0,0.350000,0.228259,0.003261,0.219101,4.095238e-02,0.148571,2000.0
"[2000.0, 3400.0)",credit.amount,242,0.242,182.0,60.0,0.247934,-0.262364,0.015742,0.219101,1.009524e-01,0.148571,3400.0
"[3400.0, 4000.0)",credit.amount,80,0.080,66.0,14.0,0.175000,-0.703300,0.033490,0.219101,1.485714e-01,0.148571,4000.0
"[4000.0, 5000.0)",credit.amount,58,0.058,31.0,27.0,0.465517,0.709148,0.032418,0.219101,1.028571e-01,0.148571,5000.0
"[5000.0, 9200.0)",credit.amount,138,0.138,89.0,49.0,0.355072,0.250482,0.009065,0.219101,6.666667e-02,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.219101,1.110223e-16,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.219101,1.110223e-16,0.148571,special


### 并行

varReport使用joblib对列的报告生成进行了并行优化,
+ 当基础的数据量非常大列较多时可通过设定n_jobs=任务数进行并行以提高运行速度
+ 一般情况下建议n_jobs=1

In [45]:
#模拟较大大数据量,3w行1000个特征
X_big=pd.DataFrame(np.random.rand(30000,1000),columns=['f'+str(i) for i in range(1000)])
y_big=pd.Series(np.random.randint(0,2,30000),name='target')
breaks_list_big={col:[0.2,0.4,0.6,0.8] for col in X_big.columns}

In [46]:
X_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1000 entries, f0 to f999
dtypes: float64(1000)
memory usage: 228.9 MB


In [47]:
%%time
res_p=bm.varReport(breaks_list_big,n_jobs=-1).fit(X_big,y_big) 

CPU times: user 845 ms, sys: 143 ms, total: 988 ms
Wall time: 5.42 s


In [48]:
%%time
res=bm.varReport(breaks_list_big,n_jobs=1).fit(X_big,y_big)

CPU times: user 13.5 s, sys: 98.6 ms, total: 13.6 s
Wall time: 13.7 s


In [49]:
#结果一致
pd.concat(res.var_report_dict).equals(pd.concat(res_p.var_report_dict))

True

## varReportSinge

### 基本用法

对单一特征产生报告可用bm.varReportSinge,其可以灵活得调整分箱

In [20]:
bm.varReportSinge().report(X['age.in.years'],y,[20,30,40,50])

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 20.0)",age.in.years,2,0.002,1.0,1.0,0.500000,0.847298,0.001614,0.064422,0.001905,0.122381,20.0
"[20.0, 30.0)",age.in.years,369,0.369,233.0,136.0,0.368564,0.308914,0.037217,0.064422,0.122381,0.122381,30.0
"[30.0, 40.0)",age.in.years,330,0.330,245.0,85.0,0.257576,-0.211309,0.014087,0.064422,0.055714,0.122381,40.0
"[40.0, 50.0)",age.in.years,174,0.174,130.0,44.0,0.252874,-0.236047,0.009217,0.064422,0.016667,0.122381,50.0
"[50.0, inf)",age.in.years,125,0.125,91.0,34.0,0.272000,-0.137201,0.002287,0.064422,0.000000,0.122381,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.064422,0.000000,0.122381,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.064422,0.000000,0.122381,missing


In [21]:
bm.varReportSinge().report(X['age.in.years'],y,[25,35,45,55])

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 25.0)",age.in.years,149,0.149,88.0,61.0,0.409396,0.480835,0.037322,0.089385,0.077619,0.131429,25.0
"[25.0, 35.0)",age.in.years,399,0.399,268.0,131.0,0.328321,0.131508,0.007076,0.089385,0.131429,0.131429,35.0
"[35.0, 45.0)",age.in.years,251,0.251,193.0,58.0,0.231076,-0.354949,0.029241,0.089385,0.049048,0.131429,45.0
"[45.0, 55.0)",age.in.years,122,0.122,94.0,28.0,0.229508,-0.363792,0.014898,0.089385,0.008095,0.131429,55.0
"[55.0, inf)",age.in.years,79,0.079,57.0,22.0,0.278481,-0.104711,0.000848,0.089385,0.000000,0.131429,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,missing


当然也支持样本权重

In [22]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index)
bm.varReportSinge().report(X['age.in.years'],y,[25,35,45,55],sample_weight)

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 25.0)",age.in.years,941,0.128904,880.0,61.0,0.064825,0.480835,0.037322,0.089385,0.077619,0.131429,25.0
"[25.0, 35.0)",age.in.years,2811,0.385068,2680.0,131.0,0.046603,0.131508,0.007076,0.089385,0.131429,0.131429,35.0
"[35.0, 45.0)",age.in.years,1988,0.272329,1930.0,58.0,0.029175,-0.354949,0.029241,0.089385,0.049048,0.131429,45.0
"[45.0, 55.0)",age.in.years,968,0.132603,940.0,28.0,0.028926,-0.363792,0.014898,0.089385,0.008095,0.131429,55.0
"[55.0, inf)",age.in.years,592,0.081096,570.0,22.0,0.037162,-0.104711,0.000848,0.089385,0.000000,0.131429,inf
special,age.in.years,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,special
missing,age.in.years,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.089385,0.000000,0.131429,missing


## varGroupsReport

在实践中,单一的特征分析报告往往仍旧无法满足实际需要,往往希望通过不同的月份\产品\客群查看各个策略的表现,那么就需要分组产生分析报告

varGroupsReport就是用来满足这种需要的

### 基本用法

In [23]:
#模拟月份(9,10,11,12月),模拟客群(3个)

X_all=X.join(y).assign(
    month=np.random.randint(9,13,y.size),
    client_group=pd.Series(np.random.randint(0,3,y.size),index=y.index).map({0:'g1',1:'g2',2:'g3'})
)

In [24]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,row_limit=0,
                          n_jobs=1).fit(X_all)

vtab_g中共五张报表:
+ report_all:各个组的特征报告
+ report_brief:各个组的简化版报告,只保留count,badprob,woe,total_iv,ks_max
+ report_count:各个组的简化版报告,只保留count
+ report_badprob:各个组的简化版报告,只保留badprob
+ report_iv:各个组的简化版报告,只保留total_iv
+ report_ks:各个组的简化版报告,只保留ks_max

以report_brief为例

In [25]:
vtab_g.report_dict['report_brief'][['variable','bin','10']] #只看10月份的报告

variable  \
                                           
                                           
0                           age.in.years   
1                           age.in.years   
2                           age.in.years   
3                           age.in.years   
4                           age.in.years   
..                                   ...   
115  status.of.existing.checking.account   
116                            telephone   
117                            telephone   
118                            telephone   
119                            telephone   

                                          bin    10                       \
                                                 g1                        
                                              count   badprob        woe   
0                                [-inf, 26.0)    25  0.520000   0.485508   
1                                [26.0, 28.0)     9  0.555556   0.628609   
2                                [28.0, 30.0)     7  0.571429   0.693147   
3                                [30.0, 35.0)    22  0.500000   0.405465   
4                                [35.0, 37.0)     8  0.000000 -20.962158   
..                                        ...   ...       ...        ...   
115                                   missing     0       NaN   0.000000   
116  yes, registered under the customers name    43  0.302326  -0.430783   
117                                      none    62  0.467742   0.276253   
118                                   special     0       NaN   0.000000   
119                                   missing     0       NaN   0.000000   

                                                                              \
                           g2                                             g3   
     total_iv    ks_max count   badprob        woe  total_iv    ks_max count   
0    2.941894  0.309524    16  0.375000   0.354172  2.075009  0.074561    15   
1    2.941894  0.309524    10  0.100000  -1.332227  2.075009  0.074561     4   
2    2.941894  0.309524     7  0.571429   1.152680  2.075009  0.074561     6   
3    2.941894  0.309524    18  0.277778  -0.090514  2.075009  0.074561    12   
4    2.941894  0.309524     3  0.000000 -20.081412  2.075009  0.074561    10   
..        ...       ...   ...       ...        ...       ...       ...   ...   
115  0.622269  0.341270     0       NaN   0.000000  2.218908  0.407895     0   
116  0.117839  0.166667    26  0.192308  -0.570087  0.127431  0.160088    35   
117  0.117839  0.166667    55  0.345455   0.225917  0.127431  0.160088    45   
118  0.117839  0.166667     0       NaN   0.000000  0.127431  0.160088     0   
119  0.117839  0.166667     0       NaN   0.000000  0.127431  0.160088     0   

                                              
                                              
      badprob        woe  total_iv    ks_max  
0    0.666667   1.600704  6.543241  0.398932  
1    0.500000   0.907557  6.543241  0.398932  
2    0.166667  -0.701881  6.543241  0.398932  
3    0.166667  -0.701881  6.543241  0.398932  
4    0.000000 -21.285385  6.543241  0.398932  
..        ...        ...       ...       ...  
115       NaN   0.000000  1.512023  0.466056  
116  0.200000  -0.478737  0.147932  0.186880  
117  0.355556   0.312850  0.147932  0.186880  
118       NaN   0.000000  0.147932  0.186880  
119       NaN   0.000000  0.147932  0.186880  

[120 rows x 17 columns]

In [26]:
vtab_g.report_dict['report_brief'] #全部报告

variable  \
                                           
                                           
0                           age.in.years   
1                           age.in.years   
2                           age.in.years   
3                           age.in.years   
4                           age.in.years   
..                                   ...   
115  status.of.existing.checking.account   
116                            telephone   
117                            telephone   
118                            telephone   
119                            telephone   

                                          bin    10                       \
                                                 g1                        
                                              count   badprob        woe   
0                                [-inf, 26.0)    25  0.520000   0.485508   
1                                [26.0, 28.0)     9  0.555556   0.628609   
2                                [28.0, 30.0)     7  0.571429   0.693147   
3                                [30.0, 35.0)    22  0.500000   0.405465   
4                                [35.0, 37.0)     8  0.000000 -20.962158   
..                                        ...   ...       ...        ...   
115                                   missing     0       NaN   0.000000   
116  yes, registered under the customers name    43  0.302326  -0.430783   
117                                      none    62  0.467742   0.276253   
118                                   special     0       NaN   0.000000   
119                                   missing     0       NaN   0.000000   

                                                    ...     9            \
                           g2                       ...    g2             
     total_iv    ks_max count   badprob        woe  ... count   badprob   
0    2.941894  0.309524    16  0.375000   0.354172  ...    17  0.411765   
1    2.941894  0.309524    10  0.100000  -1.332227  ...     9  0.222222   
2    2.941894  0.309524     7  0.571429   1.152680  ...     8  0.500000   
3    2.941894  0.309524    18  0.277778  -0.090514  ...    16  0.187500   
4    2.941894  0.309524     3  0.000000 -20.081412  ...     6  0.000000   
..        ...       ...   ...       ...        ...  ...   ...       ...   
115  0.622269  0.341270     0       NaN   0.000000  ...     0       NaN   
116  0.117839  0.166667    26  0.192308  -0.570087  ...    43  0.232558   
117  0.117839  0.166667    55  0.345455   0.225917  ...    51  0.274510   
118  0.117839  0.166667     0       NaN   0.000000  ...     0       NaN   
119  0.117839  0.166667     0       NaN   0.000000  ...     0       NaN   

                                                                        \
                                      g3                                 
           woe  total_iv    ks_max count   badprob       woe  total_iv   
0     0.713766  3.614657  0.241667    20  0.350000  0.210240  2.348853   
1    -0.182322  3.614657  0.241667     5  0.200000 -0.557015  2.348853   
2     1.070441  3.614657  0.241667     9  0.444444  0.606136  2.348853   
3    -0.395896  3.614657  0.241667    12  0.333333  0.136132  2.348853   
4   -20.569115  3.614657  0.241667     7  0.142857 -0.962480  2.348853   
..         ...       ...       ...   ...       ...       ...       ...   
115   0.000000  2.158814  0.333333     0       NaN  0.000000  0.680223   
116  -0.123481  0.012161  0.054762    27  0.370370  0.298651  0.050335   
117   0.098581  0.012161  0.054762    52  0.269231 -0.169249  0.050335   
118   0.000000  0.012161  0.054762     0       NaN  0.000000  0.050335   
119   0.000000  0.012161  0.054762     0       NaN  0.000000  0.050335   

               
               
       ks_max  
0    0.121212  
1    0.121212  
2    0.121212  
3    0.121212  
4    0.121212  
..        ...  
115  0.355303  
116  0.107576  
117  0.107576  
118  0.107576  
119  0.107576  

[120 rows x 62 columns]

In [27]:
vtab_g.report_dict['report_ks']

variable        10            \
                                                             g1        g2   
                                                         ks_max    ks_max   
0                                        age.in.years  0.309524  0.074561   
1                                       credit.amount  0.198413  0.133772   
2                                      credit.history  0.301587  0.337719   
3                                   duration.in.month  0.142857  0.269737   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.079365  0.076754   
6   installment.rate.in.percentage.of.disposable.i...  0.031746  0.245614   
7                                                 job  0.126984  0.026316   
8             number.of.existing.credits.at.this.bank  0.126984  0.002193   
9   number.of.people.being.liable.to.provide.maint...  0.007937  0.133772   
10                        other.debtors.or.guarantors  0.079365  0.070175   
11                            other.installment.plans  0.087302  0.067982   
12                            personal.status.and.sex  0.126984  0.083333   
13                           present.employment.since  0.150794  0.151316   
14                            present.residence.since  0.071429  0.098684   
15                                           property  0.190476  0.195175   
16                                            purpose  0.079365  0.291667   
17                          savings.account.and.bonds  0.071429  0.072368   
18                status.of.existing.checking.account  0.341270  0.407895   
19                                          telephone  0.166667  0.160088   

                    11                            12                      \
          g3        g1        g2        g3        g1        g2        g3   
      ks_max    ks_max    ks_max    ks_max    ks_max    ks_max    ks_max   
0   0.398932  0.140678  0.190887  0.212482  0.217593  0.038462  0.335423   
1   0.232647  0.196610  0.216749  0.218093  0.162037  0.134615  0.119122   
2   0.255530  0.065254  0.146552  0.315568  0.134259  0.153846  0.231975   
3   0.224256  0.175424  0.302956  0.262973  0.199074  0.153846  0.327586   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.171625  0.070339  0.135468  0.289621  0.199074  0.076923  0.139498   
6   0.038139  0.142373  0.129310  0.107293  0.199074  0.134615  0.164577   
7   0.138063  0.064407  0.146552  0.176017  0.143519  0.038462  0.115987   
8   0.107551  0.122881  0.118227  0.097475  0.069444  0.076923  0.043887   
9   0.132723  0.086441  0.001232  0.003506  0.134259  0.076923  0.197492   
10  0.035088  0.067797  0.072660  0.037167  0.046296  0.000000  0.040752   
11  0.010679  0.080508  0.235222  0.234923  0.041667  0.153846  0.043887   
12  0.188406  0.132203  0.075123  0.047686  0.041667  0.076923  0.130094   
13  0.257056  0.171186  0.253695  0.316971  0.125000  0.192308  0.117555   
14  0.100686  0.259322  0.039409  0.168303  0.157407  0.115385  0.034483   
15  0.282990  0.142373  0.043103  0.199158  0.222222  0.173077  0.125392   
16  0.108314  0.145763  0.102217  0.066620  0.319444  0.384615  0.172414   
17  0.405034  0.258475  0.237685  0.332398  0.291667  0.134615  0.225705   
18  0.466056  0.292373  0.283251  0.450210  0.449074  0.403846  0.346395   
19  0.186880  0.142373  0.128079  0.169705  0.166667  0.076923  0.029781   

           9                      
          g1        g2        g3  
      ks_max    ks_max    ks_max  
0   0.090909  0.241667  0.121212  
1   0.150000  0.255952  0.187879  
2   0.304545  0.176190  0.108333  
3   0.190909  0.186905  0.415152  
4   0.000000  0.000000  0.000000  
5   0.045455  0.257143  0.300000  
6   0.081818  0.157143  0.085606  
7   0.036364  0.078571  0.164394  
8   0.213636  0.052381  0.065909  
9   0.027273  0.047619  0.080303  
10  0.013636  0.015476  0.090909  
11  0.100000  0.119048  0.081061  

### 排序组特征水平

上述报告中,月份排序有一定问题,那么若希望以指定顺序排序报告则可通过sort_columns参数设定 

In [28]:
sort_columns={
    'month':['9','10','11','12'],
    'client_group':['g3','g2','g1']
} 

In [29]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,row_limit=0,
                          sort_columns=sort_columns,
                          n_jobs=1).fit(X_all)

In [30]:
vtab_g.report_dict['report_ks'] #排序后的数据

variable         9            \
                                                             g3        g2   
                                                         ks_max    ks_max   
0                                        age.in.years  0.121212  0.241667   
1                                       credit.amount  0.187879  0.255952   
2                                      credit.history  0.108333  0.176190   
3                                   duration.in.month  0.415152  0.186905   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.300000  0.257143   
6   installment.rate.in.percentage.of.disposable.i...  0.085606  0.157143   
7                                                 job  0.164394  0.078571   
8             number.of.existing.credits.at.this.bank  0.065909  0.052381   
9   number.of.people.being.liable.to.provide.maint...  0.080303  0.047619   
10                        other.debtors.or.guarantors  0.090909  0.015476   
11                            other.installment.plans  0.081061  0.119048   
12                            personal.status.and.sex  0.089394  0.078571   
13                           present.employment.since  0.151515  0.209524   
14                            present.residence.since  0.128788  0.060714   
15                                           property  0.355303  0.202381   
16                                            purpose  0.209091  0.201190   
17                          savings.account.and.bonds  0.228030  0.147619   
18                status.of.existing.checking.account  0.355303  0.333333   
19                                          telephone  0.107576  0.054762   

                    10                            11                      \
          g1        g3        g2        g1        g3        g2        g1   
      ks_max    ks_max    ks_max    ks_max    ks_max    ks_max    ks_max   
0   0.090909  0.398932  0.074561  0.309524  0.212482  0.190887  0.140678   
1   0.150000  0.232647  0.133772  0.198413  0.218093  0.216749  0.196610   
2   0.304545  0.255530  0.337719  0.301587  0.315568  0.146552  0.065254   
3   0.190909  0.224256  0.269737  0.142857  0.262973  0.302956  0.175424   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.045455  0.171625  0.076754  0.079365  0.289621  0.135468  0.070339   
6   0.081818  0.038139  0.245614  0.031746  0.107293  0.129310  0.142373   
7   0.036364  0.138063  0.026316  0.126984  0.176017  0.146552  0.064407   
8   0.213636  0.107551  0.002193  0.126984  0.097475  0.118227  0.122881   
9   0.027273  0.132723  0.133772  0.007937  0.003506  0.001232  0.086441   
10  0.013636  0.035088  0.070175  0.079365  0.037167  0.072660  0.067797   
11  0.100000  0.010679  0.067982  0.087302  0.234923  0.235222  0.080508   
12  0.240909  0.188406  0.083333  0.126984  0.047686  0.075123  0.132203   
13  0.118182  0.257056  0.151316  0.150794  0.316971  0.253695  0.171186   
14  0.304545  0.100686  0.098684  0.071429  0.168303  0.039409  0.259322   
15  0.154545  0.282990  0.195175  0.190476  0.199158  0.043103  0.142373   
16  0.390909  0.108314  0.291667  0.079365  0.066620  0.102217  0.145763   
17  0.131818  0.405034  0.072368  0.071429  0.332398  0.237685  0.258475   
18  0.486364  0.466056  0.407895  0.341270  0.450210  0.283251  0.292373   
19  0.000000  0.186880  0.160088  0.166667  0.169705  0.128079  0.142373   

          12                      
          g3        g2        g1  
      ks_max    ks_max    ks_max  
0   0.335423  0.038462  0.217593  
1   0.119122  0.134615  0.162037  
2   0.231975  0.153846  0.134259  
3   0.327586  0.153846  0.199074  
4   0.000000  0.000000  0.000000  
5   0.139498  0.076923  0.199074  
6   0.164577  0.134615  0.199074  
7   0.115987  0.038462  0.143519  
8   0.043887  0.076923  0.069444  
9   0.197492  0.076923  0.134259  
10  0.040752  0.000000  0.046296  
11  0.043887  0.153846  0.041667  

+ 若希望client_group在第一行,month在第二行,那么只需要把columns=['month','client_group']换为columns=['client_group','month']

### 使用row_limit

有时组分得过细会导致组内样本量不足,指标可能不具备代表性,因此可通过row_limit进行限定,若组数量小于row_limit时,该组就不会统计其任何指标

In [31]:
vtab_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['month','client_group'],target=y.name,
                          row_limit=80,
                          sort_columns=sort_columns,
                          n_jobs=1).fit(X_all)

group ('10', 'g3') has rows less than 80,output will return None
group ('11', 'g1') has rows less than 80,output will return None
group ('12', 'g1') has rows less than 80,output will return None
group ('12', 'g2') has rows less than 80,output will return None
group ('12', 'g3') has rows less than 80,output will return None
group ('9', 'g1') has rows less than 80,output will return None
group ('9', 'g3') has rows less than 80,output will return None


可以看到警告信息中相应剔除的组的信息

In [32]:
vtab_g.report_dict['report_ks']

variable         9        10  \
                                                             g2        g2   
                                                         ks_max    ks_max   
0                                        age.in.years  0.241667  0.074561   
1                                       credit.amount  0.255952  0.133772   
2                                      credit.history  0.176190  0.337719   
3                                   duration.in.month  0.186905  0.269737   
4                                      foreign.worker  0.000000  0.000000   
5                                             housing  0.257143  0.076754   
6   installment.rate.in.percentage.of.disposable.i...  0.157143  0.245614   
7                                                 job  0.078571  0.026316   
8             number.of.existing.credits.at.this.bank  0.052381  0.002193   
9   number.of.people.being.liable.to.provide.maint...  0.047619  0.133772   
10                        other.debtors.or.guarantors  0.015476  0.070175   
11                            other.installment.plans  0.119048  0.067982   
12                            personal.status.and.sex  0.078571  0.083333   
13                           present.employment.since  0.209524  0.151316   
14                            present.residence.since  0.060714  0.098684   
15                                           property  0.202381  0.195175   
16                                            purpose  0.201190  0.291667   
17                          savings.account.and.bonds  0.147619  0.072368   
18                status.of.existing.checking.account  0.333333  0.407895   
19                                          telephone  0.054762  0.160088   

                    11            
          g1        g3        g2  
      ks_max    ks_max    ks_max  
0   0.309524  0.212482  0.190887  
1   0.198413  0.218093  0.216749  
2   0.301587  0.315568  0.146552  
3   0.142857  0.262973  0.302956  
4   0.000000  0.000000  0.000000  
5   0.079365  0.289621  0.135468  
6   0.031746  0.107293  0.129310  
7   0.126984  0.176017  0.146552  
8   0.126984  0.097475  0.118227  
9   0.007937  0.003506  0.001232  
10  0.079365  0.037167  0.072660  
11  0.087302  0.234923  0.235222  
12  0.126984  0.047686  0.075123  
13  0.150794  0.316971  0.253695  
14  0.071429  0.168303  0.039409  
15  0.190476  0.199158  0.043103  
16  0.079365  0.066620  0.102217  
17  0.071429  0.332398  0.237685  
18  0.341270  0.450210  0.283251  
19  0.166667  0.169705  0.128079

### 产生psi报告

varGroupsReport可以产生psi报告以比较各个组中各个变量的分布变动情况

In [33]:
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],target=y.name,
                    row_limit=0,output_psi=True,n_jobs=1).fit(X_all)

In [50]:
vtabs_g.report_dict['report_psi'].head(21)

,variable,bin,g1,g2,g3
,,,count_distr,count_distr,count_distr
0,age.in.years,"[-inf, 26.0)",0.189911,0.194690,0.185185
1,age.in.years,"[26.0, 28.0)",0.100890,0.100295,0.101852
2,age.in.years,"[28.0, 30.0)",0.077151,0.079646,0.083333
3,age.in.years,"[30.0, 35.0)",0.198813,0.165192,0.166667
4,age.in.years,"[35.0, 37.0)",0.089021,0.067847,0.080247
5,age.in.years,"[37.0, 48.0)",0.207715,0.215339,0.243827
6,age.in.years,"[48.0, 53.0)",0.056380,0.061947,0.046296
7,age.in.years,"[53.0, inf)",0.080119,0.115044,0.092593
8,age.in.years,missing,0.000000,0.000000,0.000000


每一个变量的分组最后一行都会有psi用于显示各个组与基准分布比较而产生的psi

默认情况下psi的基准分布为全量数据分布,这里也可以指定某一组的分布为基准分布

+ 参数psi_base用于指定分布基准
+ 参数psi_base会在模块内部通过X.query(psi_base)传递,其语法为pd.DataFrame.query()的语法

In [35]:
#选择client_group=="g1"为psi基准分布
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],target=y.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=1).fit(X_all)

In [36]:
vtabs_g.report_dict['report_psi'].head(20) #可以看到g1的psi为0

,variable,bin,g1,g2,g3
,,,count_distr,count_distr,count_distr
0,age.in.years,"[-inf, 26.0)",0.189911,0.194690,0.185185
1,age.in.years,"[26.0, 28.0)",0.100890,0.100295,0.101852
2,age.in.years,"[28.0, 30.0)",0.077151,0.079646,0.083333
3,age.in.years,"[30.0, 35.0)",0.198813,0.165192,0.166667
4,age.in.years,"[35.0, 37.0)",0.089021,0.067847,0.080247
5,age.in.years,"[37.0, 48.0)",0.207715,0.215339,0.243827
6,age.in.years,"[48.0, 53.0)",0.056380,0.061947,0.046296
7,age.in.years,"[53.0, inf)",0.080119,0.115044,0.092593
8,age.in.years,missing,0.000000,0.000000,0.000000


### 并行

varGroupsReport使用joblib对各组的报告生成进行了并行优化,

+ 当基础的数据量非常大列较多组的数量较多时可通过设定n_jobs=任务数进行并行以提高运行速度
+ 一般情况下建议n_jobs=1

In [37]:
#模拟较大数据量,3w行1000个特征,一个组特征(5个水平)
X_big=pd.DataFrame(np.random.rand(30000,1000),columns=['f'+str(i) for i in range(1000)])
y_big=pd.Series(np.random.randint(0,2,30000),name='target')

breaks_list_big={col:[0.2,0.4,0.6,0.8] for col in X_big.columns}

X_big_all=X_big.join(y_big).assign(
    client_group=pd.Series(np.random.randint(0,5,y_big.size),index=y_big.index).map({0:'g1',
                                                                                     1:'g2',
                                                                                     2:'g3',
                                                                                     3:'g4',
                                                                                     4:'g5'
                                                                                    })
)

In [38]:
X_big_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 1002 entries, f0 to client_group
dtypes: float64(1000), int64(1), object(1)
memory usage: 229.3+ MB


In [39]:
%%time
vtabs_g_p=bm.varGroupsReport(breaks_list_big,columns=['client_group'],target=y_big.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=-1).fit(X_big_all)

CPU times: user 1min 12s, sys: 58.8 s, total: 2min 11s
Wall time: 2min 37s


In [40]:
%%time
vtabs_g=bm.varGroupsReport(breaks_list_big,columns=['client_group'],target=y_big.name,
                    row_limit=0,output_psi=True,psi_base='client_group=="g1"',n_jobs=1).fit(X_big_all)

CPU times: user 2min 30s, sys: 1min 5s, total: 3min 35s
Wall time: 3min 37s


### 样本权重

varGroupsReport提供了参数sample_weight,以在报告中加入样本权重,使用方法与varReport一致

In [41]:
sample_weight=pd.Series(y.map({0:10,1:1}),index=y.index) #假定数据经过抽样后好样本权重为10,坏样本权重为1

In [42]:
vtab_g_ws=bm.varGroupsReport(bin_tree.breaks_list,columns=['split'],target=y.name,
                       sample_weight=sample_weight,
                       row_limit=0,n_jobs=1).fit(X_all.assign(split=1))

In [43]:
vtab_g_ws.report_dict['report_all'].head(10)

variable           bin     1                                      \
                              count count_distr    good   bad   badprob   
0  age.in.years  [-inf, 26.0)  1180    0.161644  1100.0  80.0  0.067797   
1  age.in.years  [26.0, 28.0)   767    0.105068   740.0  27.0  0.035202   
2  age.in.years  [28.0, 30.0)   530    0.072603   500.0  30.0  0.056604   
3  age.in.years  [30.0, 35.0)  1275    0.174658  1220.0  55.0  0.043137   
4  age.in.years  [35.0, 37.0)   682    0.093425   670.0  12.0  0.017595   
5  age.in.years  [37.0, 48.0)  1698    0.232603  1640.0  58.0  0.034158   
6  age.in.years  [48.0, 53.0)   469    0.064247   460.0   9.0  0.019190   
7  age.in.years   [53.0, inf)   699    0.095753   670.0  29.0  0.041488   
8  age.in.years       special     0    0.000000     0.0   0.0       NaN   
9  age.in.years       missing     0    0.000000     0.0   0.0       NaN   

                                                              
        woe    bin_iv  total_iv        ks    ks_max   breaks  
0  0.528844  0.057921  0.155013  0.109524  0.131429     26.0  
1 -0.160930  0.002529  0.155013  0.093810  0.131429     28.0  
2  0.336472  0.009613  0.155013  0.122381  0.131429     30.0  
3  0.050610  0.000458  0.155013  0.131429  0.131429     35.0  
4 -0.872488  0.048610  0.155013  0.075714  0.131429     37.0  
5 -0.192126  0.007868  0.155013  0.034762  0.131429     48.0  
6 -0.784119  0.028004  0.155013  0.000952  0.131429     53.0  
7  0.009901  0.000009  0.155013  0.000000  0.131429      inf  
8  0.000000  0.000000  0.155013  0.000000  0.131429  special  
9  0.000000  0.000000  0.155013  0.000000  0.131429  missing

### 导出为excel

varGroupsReport支持对组报告导出为excel,名称为var_report.xlsx

+ 参数out_path为报告输出路径,若无此路径则模块会创建该路径
+ 参数tab_suffix代表报告名称后缀,例如tab_suffix=‘_group’时,输出报告名称为var_report_group.xlsx
+ report_dict中所有的报告都会被导出,并写为不同的sheet

In [44]:
vtabs_g=bm.varGroupsReport(bin_tree.breaks_list,columns=['client_group'],
                           target=y.name,row_limit=0,
                           output_psi=True,n_jobs=1,
                           out_path='report/',
                           tab_suffix='_client'
                          ).fit(X_all)

to_excel done
